In [25]:
# load necessary packages
import py3Dmol
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from huggingface_hub import login

import esm
from esm.utils.structure.protein_chain import ProteinChain
from esm.models.esm3 import ESM3
from esm.sdk import client
from esm.sdk.api import (
    ESMProtein,
    GenerationConfig,
)

In [20]:
# load esm model
model = ESM3.from_pretrained("esm3_sm_open_v1", device=torch.device("cuda"))

/home/pkr/miniconda3/envs/protein/lib/python3.11/site-packages/esm/pretrained.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [24]:
db_id = "1ITU"  # PDB ID corresponding to Renal Dipeptidase
chain_id = "A"  # Chain ID corresponding to Renal Dipeptidase in the PDB structure
renal_dipep_chain = ProteinChain.from_rcsb(pdb_id, chain_id)
# Alternatively, we could have used ProteinChain.from_pdb() to load a protein structure from a local PDB file

ConnectTimeout: HTTPSConnectionPool(host='files.rcsb.org', port=443): Max retries exceeded with url: /download/1ITU.pdb (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f78e3ac2b90>, 'Connection to files.rcsb.org timed out. (connect timeout=None)'))

In [ ]:
# print amino acid sequence of the protein
print(renal_dipep_chain.sequence)

In [5]:
# print the atomic coordinates of the residues
# shape (n_residues, 37, 3) (37 is # of distinct atoms that may be present across all amino acids)
print("atom37_positions shape:", renal_dipep_chain.atom37_positions.shape)
print(renal_dipep_chain.atom37_positions[:3])

ValueError: Input must be an ESMProtein or ESMProteinTensor

In [ ]:
# visualize with py3dmol library
view = py3Dmol.view(width=500, height=500)

# requires the atomic coords to be in pdb format, so we convert ProteinChain obj to PDB string
pdb_str = renal_dipep_chain.to_pdb_string()

# load the pdb string into the py3dmol view object
view.addModel(pdb_str, "pdb")

# set te style of the protein chain
view.setStyle({"cartoon": {"color": "spectrum"}})

# zoom in on protein chain
view.zoomTo()

# display protein chain
view.show()

In [ ]:
# scaffold a motif from this protein using ESM3
# 1. prompt the model w/ sequence and structure of helix-coil motif
# 2. have the model generate a larger scaffold that includes the motif

# create indices for motif
motif_inds = np.arange(123, 146)

# ProteinChain objs. can be indexed like numpy arrays to extract the sequence and coordinates of a subset of residues
motif_sequence = renal_dipep_chain[motif_inds].sequence
motif_atom37_positions = renal_dipep_chain[motif_inds].atom37_positions

print("Motif sequence:", motif_sequence)
print("Motif atom37_positions shape:", motif_atom37_positions.shape)

In [6]:
class OccupancyESM: 
    def __init__(self, model_name: str,
                 layers: list = [256],
                 activation = torch.nn.ReLU()
                 ):
        self.model = ESM3.from_pretrained(model_name).to("cuda")
        self.num_layers = len(layers)
        self.        


    def forward(self, input_sequence: str):
        x = ESMProtein(sequence=input_sequence)

   
        return output

ESM3(
  (encoder): EncodeInputs(
    (sequence_embed): Embedding(64, 1536)
    (plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_per_res_plddt_projection): Linear(in_features=16, out_features=1536, bias=True)
    (structure_tokens_embed): Embedding(4101, 1536)
    (ss8_embed): Embedding(11, 1536)
    (sasa_embed): Embedding(19, 1536)
    (function_embed): ModuleList(
      (0-7): 8 x Embedding(260, 192, padding_idx=0)
    )
    (residue_embed): EmbeddingBag(1478, 1536, mode='sum', padding_idx=0)
  )
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0): UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1536, out_features=4608, bias=False)
          )
          (out_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (q_ln): LayerNorm((1536,), eps=1e-05, el